# Data Preprocessing

Hello, in this notebook we will try to download and pre-process astronomical data. The data contains 10,000 images of galaxies (either 3x64x64 or 3x224x224), confidently labelled by Galaxy Zoo volunteers as belonging to one of four morphology classes.

GalaxyMNIST has four classes: smooth and round, smooth and cigar-shaped, edge-on-disk, and unbarred spiral. The galaxies are selected from Galaxy Zoo DECaLS Campaign A (GZD-A). 

At least 17 people must have been asked the necessary questions, and at least half of them must have answered with the given class. The class labels are therefore much more confident than from, for example, simply labelling with the most common answer to some question.



### Downloading the data.

First, let us call the python script that will let us download the data.

In [8]:
from data_import.galaxy_mnist import GalaxyMNISTHighrez

'GalaxyMNISTHighrez' class has the follwing attribuits: 
 - `root`: Specifing the dir to download the data
 - `download:` A boolean value, `True` to download the images
 - `train:` A boolean value, `True` to assign the just the training data, `False` will assgin just the testing data. 
   - `Notice:` The data has a fixed 80/20 train/test division.

In [10]:
dataset_train = GalaxyMNISTHighrez(
    root='data_import/data',
    download=True,
    train=True  # by default, or False for canonical test set
)

dataset_test = GalaxyMNISTHighrez(
    root='data_import/data',
    download=True,
    train=False  # by default, or False for canonical test set
)

---

### Reading and Visualising the data

first, let us read inputs `images` and outputs `labels` into two seperate variables

In [16]:
images_train = dataset_train.data
labels_train = dataset_train.targets

let us check the shape of the two datasets (traning, and testing)

In [11]:
print(images.shape, images.dtype)
print(labels.shape, labels.dtype)

(<PIL.Image.Image image mode=RGB size=224x224>, 1)

**Exersice1:**
Store the testing set inputs and outputs into two diffrint variables, and display thier shape.

In [18]:
### -- Code here --



In [14]:
train_image, train_label = dataset_train


ValueError: too many values to unpack (expected 2)